In [1]:
import pennylane as qml
from pennylane import numpy as np
import qemzmsepc as qem

In [2]:
# we introduced noise into the network and trained it without any error mitigation techniques
# to observe the behavior of the Loss function degradation during training.

n_qubits = 2
dev = qml.device('default.mixed', wires=n_qubits)

nqubitspaulimatrices = qem.NqubitsPauliMatrices(n_qubits)
pauli_set_of_n_qubits = nqubitspaulimatrices.get_pauli_matrices_of_n_qubits()
nqubitschannel = qem.NqubitsChannel(n_qubits, pauli_set_of_n_qubits)

nqubitspaulichannel = nqubitschannel.nqubitsrandompaulichannel(p_identity=0.85)
nqubitsdepolarizingchannel = nqubitschannel.nqubitsdepolarizingchannel(0.9)

@qml.qnode(qml.device('default.mixed', wires=n_qubits))
def train_cir_without_qem(parameters):
    qml.StronglyEntanglingLayers(weights=parameters, wires=range(2))
    qml.QubitChannel(nqubitsdepolarizingchannel, wires=[0, 1])
    qml.QubitChannel(nqubitspaulichannel, wires=[0, 1])
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))

shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
weights = np.random.random(size=shape)

def cost(x):
    return (train_cir_without_qem(x) - (-1)) ** 2

opt = qml.GradientDescentOptimizer(stepsize=0.1)
steps = 100
params = weights

for i in range(steps):
    nqubitspaulichannel = nqubitschannel.nqubitsrandompaulichannel(p_identity=0.85)
    params = opt.step(cost, params)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
print("Optimized rotation angles: {}".format(params))

Cost after step     5:  2.2989231
Cost after step    10:  0.7754341
Cost after step    15:  0.3306408
Cost after step    20:  0.1006391
Cost after step    25:  0.1000156
Cost after step    30:  0.1206770
Cost after step    35:  0.1367316
Cost after step    40:  0.0564424
Cost after step    45:  0.0284204
Cost after step    50:  0.0999624
Cost after step    55:  0.0396441
Cost after step    60:  0.0242549
Cost after step    65:  0.0560122
Cost after step    70:  0.0763092
Cost after step    75:  0.0871729
Cost after step    80:  0.0700602
Cost after step    85:  0.0325430
Cost after step    90:  0.0190190
Cost after step    95:  0.0543895
Cost after step   100:  0.0827970
Optimized rotation angles: [[[ 0.32001913  1.33916428  0.23937227]
  [ 0.82005123  2.65771552 -0.14283203]]

 [[ 0.279005    0.48305163  0.43061035]
  [ 0.38823274  0.67588869  0.48096669]]]


In [3]:
# When simulating a complex scenario of time-varying noise channels,
# it can be observed that the descent of the Loss function is relatively unstable
# in the absence of error mitigation techniques.

In [4]:
# We estimate the total noise 'p_t' using the QEMZMSEPC method,
# allowing us to incorporate noise 'p_t' into the definition of the Loss function,
# thereby mitigating the impact of noise on quantum machine learning tasks.

operations = ['RX', 'RY', 'RZ', 'RX', 'RY', 'RZ', 'CNOT', 'CNOT',
              'RX', 'RY', 'RZ', 'RX', 'RY', 'RZ', 'CNOT', 'CNOT']
shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
weights = np.random.random(size=shape)
w = weights.copy().reshape(-1)
paras = [[0, w[0]], [0, w[1]], [0, w[2]], [0, w[3]], [0, w[4]], [0, w[5]], [0, 1], [1, 0],
     [0, w[6]], [0, w[7]], [0, w[8]], [0, w[9]], [0, w[10]], [0, w[11]], [0, 1], [1, 0]]
qemzmsepc = qem.QEMZMSEPC(n_qubits)
_, p_t = qemzmsepc.qemzmsepc(operations=operations, paras=paras, dev=dev, p=0.9,
                         kraus_matrices_of_a_pauli_channel=nqubitspaulichannel)
print(p_t)

0.7221190820795362


In [5]:
@qml.qnode(qml.device('default.mixed', wires=n_qubits))
def train_cir_with_qem(parameters):
    qml.StronglyEntanglingLayers(weights=parameters, wires=range(2))
    qml.QubitChannel(nqubitsdepolarizingchannel, wires=[0, 1])
    qml.QubitChannel(nqubitspaulichannel, wires=[0, 1])
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))

shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
weights = np.random.random(size=shape)

# We modify the definition of the Loss function
# by dividing the output of the quantum circuit by the noise parameter p_t
# to obtain the expected output of the quantum circuit after error mitigation,
# and then proceed with the Loss calculation.

def cost(x):
    return ((train_cir_with_qem(x) / p_t) - (-1)) ** 2

opt = qml.GradientDescentOptimizer(stepsize=0.1)
steps = 100
params = weights
qemzmsepc = qem.QEMZMSEPC(n_qubits)

for i in range(steps):
    params = opt.step(cost, params)
    w = params.copy().reshape(-1)
    paras = [[0, w[0]], [0, w[1]], [0, w[2]], [0, w[3]], [0, w[4]], [0, w[5]], [0, 1], [1, 0],
         [0, w[6]], [0, w[7]], [0, w[8]], [0, w[9]], [0, w[10]], [0, w[11]], [0, 1], [1, 0]]
    nqubitspaulichannel = nqubitschannel.nqubitsrandompaulichannel(p_identity=0.85)
    _, p_t = qemzmsepc.qemzmsepc(operations=operations, paras=paras, dev=dev, p=0.9,
                             kraus_matrices_of_a_pauli_channel=nqubitspaulichannel)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.2649683
Cost after step    10:  0.0547809
Cost after step    15:  0.0223220
Cost after step    20:  0.0121149
Cost after step    25:  0.0076445
Cost after step    30:  0.0052879
Cost after step    35:  0.0038902
Cost after step    40:  0.0029908
Cost after step    45:  0.0023766
Cost after step    50:  0.0019376
Cost after step    55:  0.0016124
Cost after step    60:  0.0013644
Cost after step    65:  0.0011708
Cost after step    70:  0.0010166
Cost after step    75:  0.0008916
Cost after step    80:  0.0007888
Cost after step    85:  0.0007032
Cost after step    90:  0.0006312
Cost after step    95:  0.0005699
Cost after step   100:  0.0005173
Optimized rotation angles: [[[ 0.55272645  1.38395009  0.09040103]
  [ 0.00984874  1.81377291 -0.06315294]]

 [[ 0.12919937  1.28799755  0.13952135]
  [ 0.60765351  0.68838086  0.21007701]]]


In [6]:
# From the results, we can observe that the Loss function exhibits a stable decreasing trend
# after error mitigation through the QEMZMSEPC scheme,
# and ultimately, the outcome is better than without error mitigation.